## Introduction




These notebooks tell a series of *data interpretation* stories. The mechanics of getting the 
data in place are thinly veiled: The heavy lifting code is relegated to modules. These modules
have names like `NotebookModule.py` and they feature `# comments` that are intended to demystify
what is going on. So that is how a prospective scientist would be able to build from what is
shown here. 


To return to the data stories: The goal is to survey both within the OOI diaspora of sensors and
across other projects, particularly ARGO, MODIS and the ROMS model.





The initial focus is the cabled array shallow profiler. 
Need a link here. 
The shallow profilers 
consist of two connected entities: The **profiler** and the **platform**. Both generate data. 


## Context of OOI

### What is OOI?


OOI stands for Ocean Observatories Initiative. It is a collection of **arrays** which are distributions
of scientific instruments. To date there are six arrays: Four in the northern hemisphere and two in 
the southern ocean. The southern ocean arrays operated for a few years but are now discontinued. 


### What is the agenda of this repository?


Agenda: Demonstrate data interpretation; first as a **technical process** and
second in terms of ocean science. 



### What are technical details to watch out for?


#### Open and subset a NetCDF data file via the `xarray Dataset`  

- Convert the principal dimension from `obs` or `row` to `time` 
    - `obs/row` are generic terms with values running 1, 2, 3... (hinders combining files into longer time series)
- Re-name certain data variables for easier use; and delete anything that is not of interest
- Identify the time range of interest
- Write a specific subset file
    - For example: Subset files that are small can live within the repo

```
# This code runs 'one line at a time' (not as a block) to work iteratively through the process
ds = xr.open_dataset(filename)
ds                                         # notice the output will show dimension as "row", "time" as a data variable


ds = ds.swap_dims({'row': 'time'})
ds = ds.rename({'some_ridiculously_long_data_variable_name':'temperature'})
ds = ds.drop('some_data_variable_that_has_no_interest_at_this_point')

ds.z.plot()                                # this produces a simple chart showing gaps in the data record

ta0 = dt64_from_doy(2021, 60)              # these time boundaries are set iteratively...
ta1 = dt64_from_doy(2021, 91)              #   ...to focus in on a particular time range with known data...
ds.sel(time=slice(ta0,  ta1)).z.plot()     #   ...where this plot is the proof


ds.sel(time=slice(ta0,  ta1)).to_netcdf(outputfile)
```




### Data products


The 
[OOI Data Catalog Documentation](https://dataexplorer.oceanobservatories.org/help/overview.html#data-products) 
describes three levels of data product, summarized: 


* Level 1 ***Instrument deployment***: Unprocessed, parsed data parameter that is in instrument/sensor 
units and resolution. See note below defining a *deployment*. This is not data we are interested in using, as a rule.


* Level 1+ ***Full-instrument time series***: A join of recovered and telemetered 
streams for non-cabled instrument deployments. For high-resolution cabled and recovered data, this product is 
binned to 1-minute resolution to allow for efficient visualization and downloads for users that do not need 
the full-resolution, gold copy (Level 2) time series. We'd like to hold out for 'gold standard'.


* Level 2 ***Full-resolution, gold standard time series***: The calibrated full-resolution dataset 
(scientific units). L2 data have been processed, pre-built, and served 
from the OOI system to the 
[OOI Data Explorer](https://dataexplorer.oceanobservatories.org/)
and to Users. The mechanisms are THREDDS and ERDDAP; file format  
NetCDF-CF. There is one file for every instrument, stream, and deployment.  For more refer to this
[Data Download](https://dataexplorer.oceanobservatories.org/help/overview.html#download-data-map-overview) link.



## Terminology



- **instrument**: A physical device with one or more sensors.
- **stream**: Sensor data.
- **deployment**: The act of putting infrastructure in the water, or the length of 
time between a platform going in the water and being recovered and brought back to shore.There are 
multiple deployment files per instrument. 



### Shallow profiler overview



The deployments run 2015 to present; with intermittent interruptions due to servicing and *force majeure*. 
There are three shallow profilers:


1. 500m depth: Outer edge of the continental shelf off of central Oregon
2. 3100m depth: Further out at the base of the continental shelf off central Oregon
3. 2100m depth: At the Juan de Fuca plate boundary at the base of Axial Seamount


#### Platform


The shallow profiler **platform** is tethered to
the sea floor by means of two long cables. It is positively buoyant and sits 200 meters below 
the surface. The platform has both power and a data connection back to shore. 


#### Profiler


The shallow profiler **profiler** rests in a cradle on the **platform**. 
It is also positively buoyant. Under normal circumstances
this profiler is allowed to rise to near the surface (depth of approximately 10 meters) nine times each day. 
This is accomplished by means of a single cable on a winch. As the profiler ascends its "upward facing"
sensors acquire data. Once the profiler reaches the top of the profile it is winched back down again. 

From notebook **Ocean 01 D** which characterizes profile ascent/descent/rest intervals: 


```
Mean time in minutes for...

Ascent:    67
Descent:   45           2 of each day's 9 descents have built-in equilibration stops that extend their duration
Rest:      45
```


#### Shallow profiler + platform data considerations


Data are acquired by the basic instruments / sensors on a continuous basis. This means that 
data acquired on the *platform* can be compared to parked *profiler* data. (See related notebook.)


The profiler
rises through and disturbs the water on ascent; and then is winched back down. Ascent data are
considered more pristine; although pH and pCO2 are unique in that they are recorded on *descent*.
(There is an ascent / descent comparison notebook.)


The table below shows data available from the profiler and its 200m-depth retaining platform. 
Pressure, density, salinity, temperature and depth are interrelated. In particular, pressure 
in decibars and depth in meters are very nearly the same. Charts of sensor value against depth 
effectively treat profiles as "instantaneous" snapshots of upper water column. 


To help economize the code I have implemented single-letter codes for the various data types.


```
Temporary fixed width font section should be replaced with a table


Abbrev   Instrument / sensor                     Profiler                         Platform
Letter
PA       CTD / Pressure                          Yes                              Yes
K        CTD / Density                           Yes                              Yes
Z        CTD / Depth                             Yes                              Yes
T        CTD / Temperature                       Yes                              Yes
S        CTD / Salinity                          Yes                              Yes
O        CTD / Dissolved Oxygen                  Yes                              ?
A        Fluorometer / Chlorophyll-A             Yes                              ?
B        Fluorometer / Backscatter               Yes                              ?
C        Fluorometer / CDOM / FDOM               Yes                              ?
H        pH                                      Yes                              No
R        pCO2                                    Yes                              ?
N        nitrate                                 Yes                              ?
ND       nitrate - dark counts                   Yes                              ?
P        PAR                                     Yes                              ?
L        Spectrophotometer: 83 channels          Yes                              No
I        Spectral Irradiance: 7 channels         Yes                              ?
UE,UN,UU Current east / north / up               Yes                              No
DE,DN,DU ADCP                                    No                               Yes
```